# Data Preprocessing,Cleansing and Merging

## We have Two seperate data file as of now
 1. overall consolidated data(rows = 16408095,columns = 59),going forward we will refer it as **consol_df**.
 2. credit card data(rows = 5470942, columns = 7), we will refer it as **credit_df**

#### First will start with consol_df into which will drop few columns which has high NA's and which is not important for our analysis.

In [1]:
%%time
# Reading the Overall consolidated file
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Bilad Retail EDA").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","14g").getOrCreate()
consol_df = spark.read.parquet("consolidated_08_nov/consolidated_08.parquet")

22/11/09 09:17:27 WARN Utils: Your hostname, amolmurme resolves to a loopback address: 127.0.1.1; using 192.168.1.9 instead (on interface wlp3s0)
22/11/09 09:17:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/09 09:17:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


CPU times: user 476 ms, sys: 974 ms, total: 1.45 s
Wall time: 7.11 s


In [2]:
# Check the count of Rows and columns
print(f' Rows = {consol_df.count()} and columns = {len(consol_df.columns)}')

 Rows = 16408095 and columns = 59


In [3]:
print(consol_df.columns,len(consol_df.columns))

['contract_no', 'sector_2', 'residence', 'customer_id', 'amount', 'installment', 'value_date', 'fin_mat_date', 'category', 'drawdown_net_amt', 'total_profit', 'rate', 'period', 'month_profit', 'salary_amount', 'occupation', 'company_name', 'city', 'sector_19', 'employment_date', 'pd_amount', 'outstanding_finance', 'days', 'x1st_pd_creation_date', 'product_description', 'asset_price', 'balloon_payment', 'birth_date', 'gender', 'martial', 'education_level', 'id', 'a_c_open_date', 'x1st_installment_date', 'service_period_yearly', 'sector_21', 'pd_days', 'down_payment', 'salary_assignment', 'occupation_type', 'remaining_inst', 'remaining_profit_amt', 'simah_score', 'behaviuor_score', 'application_score', 'no_of_dependents', 'x43', 'seller_branch', 'sector_22', 'old_maturity_date', 'last_postpone_execution_date', 'last_postpone_by_no_of_months', 'total_no_of_times_postpone_executed', 'old_maturity_date_before_postpone', 'last_installment_amt_postponed', 'nationality', 'id_date', 'dpd_create

Drop the columns having High NA's and not so necessary for our analysis based on the EDA done previously.

In [4]:
drop_cols=["sector_22","sector_21", "a_c_open_date", "days", "sector_19", "x43", "seller_branch", "x1st_pd_creation_date", "balloon_payment", "old_maturity_date_before_postpone", "last_postpone_execution_date", "last_postpone_by_no_of_months", "total_no_of_times_postpone_executed","fwd_def_tag"]
consol_df=consol_df.drop(*drop_cols)
print(consol_df.columns,len(consol_df.columns))

['contract_no', 'sector_2', 'residence', 'customer_id', 'amount', 'installment', 'value_date', 'fin_mat_date', 'category', 'drawdown_net_amt', 'total_profit', 'rate', 'period', 'month_profit', 'salary_amount', 'occupation', 'company_name', 'city', 'employment_date', 'pd_amount', 'outstanding_finance', 'product_description', 'asset_price', 'birth_date', 'gender', 'martial', 'education_level', 'id', 'x1st_installment_date', 'service_period_yearly', 'pd_days', 'down_payment', 'salary_assignment', 'occupation_type', 'remaining_inst', 'remaining_profit_amt', 'simah_score', 'behaviuor_score', 'application_score', 'no_of_dependents', 'old_maturity_date', 'last_installment_amt_postponed', 'nationality', 'id_date', 'dpd_created', 'def_flag'] 46


In [5]:
from pyspark.sql.functions import col,isnan,when,count
df_Columns=["dpd_created","def_flag","pd_days"]
consol_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]
   ).show()

+-----------+--------+--------+
|dpd_created|def_flag| pd_days|
+-----------+--------+--------+
|   14541961|       0|15898097|
+-----------+--------+--------+



In [15]:
consol_df.printSchema()

root
 |-- contract_no: string (nullable = true)
 |-- id_date: timestamp (nullable = true)
 |-- sector_2: string (nullable = true)
 |-- residence: string (nullable = true)
 |-- customer_id: double (nullable = true)
 |-- amount: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- value_date: string (nullable = true)
 |-- fin_mat_date: string (nullable = true)
 |-- category: integer (nullable = true)
 |-- drawdown_net_amt: double (nullable = true)
 |-- total_profit: double (nullable = true)
 |-- rate: double (nullable = true)
 |-- period: integer (nullable = true)
 |-- month_profit: double (nullable = true)
 |-- salary_amount: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- employment_date: string (nullable = true)
 |-- pd_amount: double (nullable = true)
 |-- outstanding_finance: double (nullable = true)
 |-- product_description: string (nullable = true)
 |-- asset

# Portfolio Segmentation

In [6]:
consol_df.select('category').distinct().count() # Find out total distinct category

14

segmentation will be done on the mapping given by ganesh which uses 'category' column.

personal finance = category(21059,21060,**21063**,21064,21065,21066,21072)>>2103>>product description>>
1. LSF Against Real Estate Mortgage
2. LSFAgainstRealEstateMortgageTOPUP"

Auto Lease	= category(21067)

Real Estate = category(21061,21062,**21063**,21069,21073)2103>>product description>>
1. Self Construction Finance
2. Topup Construction Finance"

In [15]:
from pyspark.sql.functions import col, when,udf
from pyspark.sql. types import StringType,IntegerType
personal_finance = [21059,21060,21064,21065,21066,21072]
Auto_Lease	= [21067]
Real_Estate = [21061,21062,21069,21073]

# def portfolio_segmentation(category):
#     if category in personal_finance:   
#         return "personal finance"
#     elif category in Auto_Lease:
#         return "auto lease"
#     elif category in Real_Estate:
#         return "real estate"


# fun = udf(portfolio_segmentation,StringType())
# consol_df = consol_df.withColumn("portfolio",fun(consol_df.category))

# condition_PF = (col("category").isin(personal_finance))
# condition_AL = (col("category").isin(Auto_Lease))
# condition_RE = (col("category").isin(Real_Estate))

consol_df = consol_df.withColumn('portfolio',
 when(col('category').isin(personal_finance),'personal finance')
.when(col('category').isin(Auto_Lease),'auto lease')
.when(col('category').isin(Real_Estate),'real estate')
.when((col('category') == 21063) & ((col('product_description') == 'LSF Against Real Estate Mortgage')|(col('product_description') == 'LSFAgainstRealEstateMortgageTOPUP')),'personal finance')
.when((col('category') == 21063) & ((col('product_description') == 'Self Construction Finance')|(col('product_description') == 'Topup Construction Finance')),'real estate')

)
#consol_df = consol_df.withColumn('portfolio',condition)
#consol_df = consol_df.withColumn("portfolio", when(condition_PF, "personal finance")).withColumn("portfolio", when(condition_AL, "auto lease")).withColumn("portfolio", when(condition_RE, "real estate"))



#consol_df = consol_df.withColumn("portfolio", (when(condition_PF, "personal finance") |when(condition_AL, "auto lease")|when(condition_RE, "real estate")))

In [16]:
consol_df.groupBy('portfolio').count().show()

+----------------+--------+
|       portfolio|   count|
+----------------+--------+
|     real estate| 1766111|
|            null|  356728|
|personal finance|13231831|
|      auto lease| 1053425|
+----------------+--------+



In [49]:
consol_df.groupBy('category').count().show()

+--------+-------+
|category|  count|
+--------+-------+
|   21060|  36251|
|   21061| 834765|
|   21063|1413397|
|   21067|1053425|
|   21065|1802152|
|   21072|3503804|
|   21068|     30|
|   21062|   6456|
|   21066|   8189|
|   21064|7614446|
|   21059|    123|
|   21073|  52597|
|   21069|  81602|
|   21058|    858|
+--------+-------+



In [13]:
consol_df.groupBy('product_description').count().show(truncate=False)

+-----------------------------------+-------+
|product_description                |count  |
+-----------------------------------+-------+
|Real Estate Ijara                  |6449   |
|Auto Lease                         |1053429|
|Motor Vehicle Finance              |168    |
|Buy Out Finance                    |368523 |
|Auto Lease Portfolio               |30     |
|Staff loan without profit          |35620  |
|Re-Finance Re-Shced                |49021  |
|Car Financing                      |8017   |
|Advance Salary                     |276    |
|Re- Finance by LSF                 |1611817|
|Real Estate Financing              |317111 |
|LSF Against Real Estate Mortgage   |233172 |
|Personal Loan Top Up Finance       |3110665|
|Local Share Murabaha Finance       |6694473|
|Real Estate TopUp Financing        |511330 |
|LSF ReDesign                       |491    |
|Off Plan (Fix Installment)         |28020  |
|LSFAgainstRealEstateMortgageTOPUP  |33711  |
|Self Construction Finance        

## Lets read the credit card data 

We have to add 'portfolio' column and also need to current_default_tagging based on Days. this will be contract_no level default tagging.

Once this is done we can append the credit card data with consolidated data and then will do customer level default tagging.

In [17]:
credit_df = spark.read.csv('CC_base_data_28Oct2022.csv',sep='\t',inferSchema=True,header=True,escape="\"")

In [18]:
credit_df.printSchema()

root
 |-- CIF: integer (nullable = true)
 |-- TOTAL_OUTSTANDING_BALANCE: double (nullable = true)
 |-- Days: integer (nullable = true)
 |-- CARD_PRODUCT: string (nullable = true)
 |-- Limit: double (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Contract.ID: long (nullable = true)



Total number of rows:

In [19]:
credit_df.count()

5470942

## Find count for empty, None, Null, Nan with string literals.

In [24]:
# Rename the 'Contract.ID' as contract_no
credit_df = credit_df.withColumnRenamed('Contract.ID','contract_no')

In [25]:
from pyspark.sql.functions import col,isnan,when,count
columns = ['CIF',
 'TOTAL_OUTSTANDING_BALANCE',
 'Days',
 'CARD_PRODUCT',
 'Limit',
 'contract_no']
df2 = credit_df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in columns])
df2.show()

+---+-------------------------+-----+------------+-----+-----------+
|CIF|TOTAL_OUTSTANDING_BALANCE| Days|CARD_PRODUCT|Limit|contract_no|
+---+-------------------------+-----+------------+-----+-----------+
|  0|                        0|69213|     1847969| 2480|    1162090|
+---+-------------------------+-----+------------+-----+-----------+



In [26]:
from pyspark.sql.functions import countDistinct
credit_df.select(countDistinct('Date')).show()

+--------------------+
|count(DISTINCT Date)|
+--------------------+
|                 105|
+--------------------+

